In [1]:
import os
import cv2
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
folder_path = "./images"

# List PNG files in the folder
png_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]

# Load images using OpenCV
images = []
images_path=[]
for file in png_files:
    image_path = os.path.join(folder_path, file)
    image = cv2.imread(image_path)
    # Check if image is loaded correctly
    if image is None:
        print(f"Failed to load image: {image_path}")
    else:
        print(f"Successfully loaded image: {image_path}")
        images.append(image)
        images_path.append(image_path)

Successfully loaded image: ./images\desk_2_19.png
Successfully loaded image: ./images\desk_2_19_depth.png
Successfully loaded image: ./images\desk_2_20.png
Successfully loaded image: ./images\desk_2_20_depth.png
Successfully loaded image: ./images\desk_2_21.png
Successfully loaded image: ./images\desk_2_21_depth.png
Successfully loaded image: ./images\desk_2_22.png
Successfully loaded image: ./images\desk_2_22_depth.png
Successfully loaded image: ./images\desk_2_23.png
Successfully loaded image: ./images\desk_2_23_depth.png
Successfully loaded image: ./images\desk_2_24.png
Successfully loaded image: ./images\desk_2_24_depth.png


In [5]:
base_path = "./images"
start_idx = 19
end_idx = 24
voxel_size = 0.02
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5

intrinsic = o3d.camera.PinholeCameraIntrinsic(
    o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)


def load_rgbd(index):
    color = o3d.io.read_image(f"{base_path}/desk_2_{index}.png")
    depth = o3d.io.read_image(f"{base_path}/desk_2_{index}_depth.png")
    return o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=1.5, convert_rgb_to_intensity=False)

def create_point_cloud(rgbd):
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)
    pcd.estimate_normals()
    return pcd

def pairwise_registration(source, target):
    print("Running pairwise point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine, icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine, transformation_icp)
    #TransformationEstimationPointToPlane() specifies the type of ICP. The point-to-plane method aligns the point clouds based on the closest points' normals, rather than just point-to-point distances. This is more accurate when the point clouds have smooth surfaces and you want to align them based on surface geometry.
    
    return transformation_icp, information_icp

def full_registration(pcds):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)

    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            print(f"Registering pair {start_idx + source_id} → {start_idx + target_id}")
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])

            if target_id == source_id + 1:  
                odometry = transformation_icp @ odometry
                pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(
                    source_id, target_id, transformation_icp, information_icp, uncertain=False))
            else:  # loop closure
                pose_graph.edges.append(o3d.pipelines.registration.PoseGraphEdge(
                    source_id, target_id, transformation_icp, information_icp, uncertain=True))

    return pose_graph

def integrate_rgbd_frames(rgbd_images, pose_graph):
    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=voxel_size / 2.0,
        sdf_trunc=0.04,
        color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

    for i in range(len(rgbd_images)):
        print(f"Integrating frame {start_idx + i}")
        pose = pose_graph.nodes[i].pose
        volume.integrate(rgbd_images[i], intrinsic, np.linalg.inv(pose))

    return volume.extract_point_cloud()





An information matrix is a measure of how well the points from two point clouds correspond to each other during a registration process like ICP. It's essentially used to represent the uncertainty of the alignment or transformation between two point clouds. The higher the value in the information matrix, the more confident the registration is in aligning corresponding points. This matrix is especially useful when performing global registration or non-rigid transformations, where the system needs to take into account uncertainty and error.

In [7]:
print("Loading RGBD frames...")
rgbd_images = [load_rgbd(i) for i in range(start_idx, end_idx + 1)]
pcds = [create_point_cloud(rgbd) for rgbd in rgbd_images] #indivisual pcd list

print("\nBuilding pose graph using pairwise ICP...")
pose_graph = full_registration(pcds)

print("\nOptimizing pose graph...")
o3d.pipelines.registration.global_optimization(
    pose_graph,
    o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
    o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
    o3d.pipelines.registration.GlobalOptimizationOption(
        max_correspondence_distance=max_correspondence_distance_fine,
        edge_prune_threshold=0.25,
        reference_node=0))

print("\nIntegrating RGBD frames to create final point cloud...")
pcd_combined = integrate_rgbd_frames(rgbd_images, pose_graph)



Loading RGBD frames...

Building pose graph using pairwise ICP...
Registering pair 19 → 20
Running pairwise point-to-plane ICP
Registering pair 19 → 21
Running pairwise point-to-plane ICP
Registering pair 19 → 22
Running pairwise point-to-plane ICP
Registering pair 19 → 23
Running pairwise point-to-plane ICP
Registering pair 19 → 24
Running pairwise point-to-plane ICP
Registering pair 20 → 21
Running pairwise point-to-plane ICP
Registering pair 20 → 22
Running pairwise point-to-plane ICP
Registering pair 20 → 23
Running pairwise point-to-plane ICP
Registering pair 20 → 24
Running pairwise point-to-plane ICP
Registering pair 21 → 22
Running pairwise point-to-plane ICP
Registering pair 21 → 23
Running pairwise point-to-plane ICP
Registering pair 21 → 24
Running pairwise point-to-plane ICP
Registering pair 22 → 23
Running pairwise point-to-plane ICP
Registering pair 22 → 24
Running pairwise point-to-plane ICP
Registering pair 23 → 24
Running pairwise point-to-plane ICP

Optimizing pose gr

In [9]:
pcd_combined.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
o3d.visualization.draw_geometries([pcd_combined])

In [11]:
def segment_plane(pcd, distance_threshold=0.01, ransac_n=3, num_iterations=1000):
    # Perform plane segmentation
    plane_model, inlier_indices = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)
   
    # Extract inlier points (points on the plane)
    inlier_cloud = pcd.select_by_index(inlier_indices)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])
    # Extract outlier points (points not on the plane)
    outlier_cloud = pcd.select_by_index(inlier_indices, invert=True)
    
    return inlier_cloud, outlier_cloud, plane_model

In [13]:
inlier_cloud, outlier_cloud, plane_model = segment_plane(pcd_combined)



In [15]:
print("Plane model: ", plane_model)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane model:  [0.01924708 0.78819605 0.61512318 0.53027395]


In [17]:
o3d.visualization.draw_geometries([ outlier_cloud])

In [19]:
print("Plane model: ", plane_model)
outlier_cloud1=outlier_cloud.paint_uniform_color([0.0, 0.0, 0.0])
o3d.visualization.draw_geometries([ outlier_cloud1])

Plane model:  [0.01924708 0.78819605 0.61512318 0.53027395]


In [21]:
def dbscan_clustering(pcd, eps=0.05, min_points=10):
    # Convert point cloud to numpy array
    points = np.asarray(pcd.points)
    
    # Apply DBSCAN clustering using sklearn
    from sklearn.cluster import DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_points).fit(points)
    
    # Labels for each point in the point cloud
    labels = db.labels_
    
    # Create a list of point clouds for each cluster
    clusters = []
    unique_labels = np.unique(labels)
    
    for label in unique_labels:
        for label in unique_labels:
        # Create a point cloud for each cluster (including noise points labeled as -1)
            cluster_points = points[labels == label]
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
        
        # Optionally, color the points in the cluster
        if label == -1:
            cluster_pcd.paint_uniform_color([0, 0, 0])  # Noise points (optional, can change color)
        else:
            # Assign a random color to each cluster
            cluster_pcd.paint_uniform_color(np.random.rand(3))
        
        clusters.append(cluster_pcd)
    
    return clusters



In [29]:
clusters = dbscan_clustering(outlier_cloud, eps=0.05, min_points=10)


o3d.visualization.draw_geometries(clusters)

In [ ]:
import matplotlib.pyplot as plt
pcd=outlier_cloud

labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

max_label = labels.max()
print(f"Point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd])



In [33]:
import matplotlib.pyplot as plt
pcd=outlier_cloud

labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=5, print_progress=True))

max_label = labels.max()
print(f"Point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd])


Point cloud has 21 clusters


In [35]:
#clusters as different point clouds

clusters = []
for i in range(max_label + 1):
    cluster_points = pcd.select_by_index(np.where(labels == i)[0])
    clusters.append(cluster_points)
    o3d.visualization.draw_geometries([cluster_points])

for i, cluster in enumerate(clusters):
    o3d.io.write_point_cloud(f"cluster_{i}.ply", cluster)